# Detección de crecimiento urbano con imagenes del satélite Sentinel - 1

## 2_ Predicción

Este notebook genera la prediccion del modelo sobre un nuevo dataset. Para ello primero descargamos las imagenes satelitales y luego con satproc generamos el dataset de predicción con el formato necesario para el modelo de ML


**Definicion de los parametros para la predicción**

In [36]:
from unetseg.predict import PredictConfig, predict
from unetseg.evaluate import plot_data_results
import subprocess
import os
import pandas as pd
import glob

In [28]:
CSV_RESULTS_FILE = "buckets_v3_1_resultados.csv"
CSV_IMG_S2_FILE  = "buckets_v3_1_images-s1.csv"
    
BANDS = "VHd_VVd_VHdVVd"
YEAR  = "2021"
BASE_PATH = "../"

#model
SIZE      = 100
STEP_SIZE = 100

MODEL = f'UNet_TEST_160x160_spe100_3N_spe100_colombia_atlantico.h5'


CONDITION = True
ZONA = "pacifico"

### Descarga de las imagenes de predicción

Descarga las imagenes del bucket

### Satproc

Con esta herramienta genereamos un dataset de imágenes de predicción que, a diferencia del de entrenamiento, solo contiene imágenes.

In [31]:
def execute_satproc_function(SIZE,STEP_SIZE,BASE_PATH,img_folder,YEAR,DPTO_COD,COD_MUNICIPIO):
   
   
    path_to_files  = f'{img_folder}/{COD_MUNICIPIO}/*.tif'
    dataset_folder_ = f'{BASE_PATH}/dataset/data_predict/province_{COD_MUNICIPIO}_{DPTO_COD}_year{YEAR}/{str(SIZE)}_{str(STEP_SIZE)}/'
  
    print("path_to_files-->",path_to_files)
    print("dataset_folder_-->",dataset_folder_)
   
    !satproc_extract_chips $path_to_files \
                    -o $dataset_folder_ \
                    --size $SIZE \
                    --step-size $STEP_SIZE \
                    --rescale \
                    --rescale-mode values --min -15 --max 5
    return dataset_folder_

Pasamos la ruta de las imágenes de predicción y la ruta donde queremos que se guarde el resultado.



## Prediccion

Definimos la configuración para la predicción. Debemos pasar la ruta de las imágenes en el dataset de predicción, que es la ruta de salida de la línea anterior, y la correspondiente al modelo que entrenamos.

In [34]:
def excetute_prediction (dataset_folder,BASE_PATH,YEAR,SIZE,STEP_SIZE,MODEL,DPTO_COD,COD_MUNICIPIO,BUCKET_RESULTS):
    predict_config = PredictConfig(
         
            images_path   = dataset_folder, # ruta a las imagenes sobre las cuales queremos predecir
            results_path  = f'{BASE_PATH}/dataset/data_results/province_{COD_MUNICIPIO}_{DPTO_COD}_year{YEAR}/{str(SIZE)}_{str(STEP_SIZE)}/'
, # ruta de destino para nuestra predicción
            batch_size  = 16,
            model_path  = os.path.join(BASE_PATH +'data/weights', MODEL),  #  ruta al modelo (.h5)
            height = 160,
            width  = 160,
            n_channels = 3,
            n_classes  = 1,
            class_weights = [1])

    #A continuación ejecutamos la predicción
    predict(predict_config)    
    #plot_data_results(num_samples=2, fig_size=(2, 2), predict_config=predict_config, img_ch =2, n_bands=3)

    #create VTR
    !rm tmp_list.txt

    vrt_path        = f'{predict_config.results_path}/../{COD_MUNICIPIO}_{YEAR}.vrt' # ruta de destino para nuestra predicción
    tif_path        = f'{predict_config.results_path}/../{COD_MUNICIPIO}_{YEAR}.tif'
    predict_results = f'{predict_config.results_path}/*tif' # ruta de destino para nuestra predicción

    filenames = glob.glob(predict_results)

    with open('tmp_list.txt', 'w') as f:
        for line in filenames:
            f.write(line)
            f.write('\n')

    !gdalbuildvrt -input_file_list tmp_list.txt $vrt_path
    
    !gdal_translate -of GTiff -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 $vrt_path $tif_path
    
    # upload to a bucket
    !gsutil -m cp -r  $tif_path $BUCKET_RESULTS
    

## RUN

In [39]:
# load the data with pd.read_csv
df = pd.read_csv(CSV_IMG_S2_FILE).reset_index()
df_results = pd.read_csv(CSV_RESULTS_FILE).reset_index()

df.sort_values(by=["PRIORIDAD"], ascending=True)

if CONDITION:
    df[df["REGION"] == ZONA]

for index, row in df.iterrows():
   
    COD_MUNICIPIO = row['COD_MUNICIPIO']
    NOM_MUNICIPIO = row['NOM_MUNICIPIO']
    DPTO_COD      = row['DPTO_COD']
    DPTO_NAME     = row['DPTO_NAME']
    PRIORIDAD     = row['PRIORIDAD']
    REGION        = row['REGION']
    BUCKET_IMG    = row['BUCKET']
    
    print ('=================================================================================================')
    print ( f'Working with {NOM_MUNICIPIO} - {DPTO_NAME}, corresponding with COD_MUNICIPIO {COD_MUNICIPIO} and DPTO_COD {DPTO_COD}' )
    print ('=================================================================================================')

    print ( f'priority number: {PRIORIDAD}')
    
    ############################
    ## Download images
    ###########################
    print ("Download images...")
    img_folder = f'{BASE_PATH}/images_s1_{BANDS}/{YEAR}/'
    !mkdir -p $img_folder
    !gsutil -m cp -r -n $BUCKET_IMG  $img_folder
    
    ############################
    ## Satproc  & predict 
    ###########################
    
    #read bucket
    BUCKET_RESULTS = df_results[ df_results["COD_MUNICIPIO"] == COD_MUNICIPIO].BUCKET
    print (BUCKET_RESULTS.values[0])
    #satproc
    dataset_folder = execute_satproc_function(SIZE,STEP_SIZE,BASE_PATH,img_folder,YEAR,DPTO_COD,COD_MUNICIPIO)
    #predict
    excetute_prediction (dataset_folder,BASE_PATH,YEAR,SIZE,STEP_SIZE,MODEL,DPTO_COD,COD_MUNICIPIO,BUCKET_RESULTS)
    

Working with Necoclí - Antioquia, corresponding with COD_MUNICIPIO 5490 and DPTO_COD 5
priority number: 99
Download images...
Skipping existing item: file://..//images_s1_VHd_VVd_VHdVVd/2021/5490/5490_2021.tif
gs://dym-indunor-temp/immap/v3_1/results/5/5490/
path_to_files--> ..//images_s1_VHd_VVd_VHdVVd/2021//5490/*.tif
dataset_folder_--> ..//dataset/data_predict/province_5490_5_year2021/100_100/
Rasters: 100%|####################################| 1/1 [00:00<00:00, 14.33it/s]
Total images to predict (..//dataset/data_predict/province_5490_5_year2021/100_100/images/*.tif): 3016
After skipping existing results: 0


0it [00:00, ?it/s]

Done!


0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 5710, 5102
0...10...20...30...40...50...60...70...80...90...100 - done.
CommandException: No URLs matched: 0
Copying file://..//dataset/data_results/province_5490_5_year2021/100_100//../5490_2021.tif [Content-Type=image/tiff]...
- [1/1 files][  7.7 MiB/  7.7 MiB] 100% Done                                    
Operation completed over 1 objects/7.7 MiB.                                      
CommandException: 1 file/object could not be transferred.
/bin/bash: line 1: Name:: command not found
Working with Barranquilla - Atlantico, corresponding with COD_MUNICIPIO 8001 and DPTO_COD 8
priority number: 1
Download images...
Skipping existing item: file://..//images_s1_VHd_VVd_VHdVVd/2021/8001/8001_2021.tif
gs://dym-indunor-temp/immap/v3_1/results/8/8001/
path_to_files--> ..//images_s1_VHd_VVd_VHdVVd/2021//8001/*.tif
dataset_folder_--> ..//dataset/data_predict/province_8001_8_year2021/100_100/
Rasters: 100%|########

0it [00:00, ?it/s]

...

Done!
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 2158, 3764
0...10...20...30...40...50...60...70...80...90...100 - done.
CommandException: No URLs matched: 75
Copying file://..//dataset/data_results/province_76890_76_year2021/100_100//../76890_2021.tif [Content-Type=image/tiff]...
/ [1/1 files][  2.7 MiB/  2.7 MiB] 100% Done                                    
Operation completed over 1 objects/2.7 MiB.                                      
CommandException: 1 file/object could not be transferred.
/bin/bash: line 1: Name:: command not found
Working with Yumbo - Valle del Cauca, corresponding with COD_MUNICIPIO 76892 and DPTO_COD 76
priority number: 76
Download images...
Copying gs://dym-indunor-temp/immap/v3_1/images/76/76892/76892_2021.tif...
- [1/1 files][ 65.8 MiB/ 65.8 MiB] 100% Done                                    
Operation completed over 1 objects/65.8 MiB.                                     
gs://dym-indunor-temp/immap/v3_1/results/76/76

100%|████████████████████████████████████████████████████████████████| 29/29 [00:46<00:00,  1.60s/it]


Done!
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1857, 2322
0...10...20...30...40...50...60...70...80...90...100 - done.
CommandException: No URLs matched: 76
Copying file://..//dataset/data_results/province_76892_76_year2021/100_100//../76892_2021.tif [Content-Type=image/tiff]...
/ [1/1 files][  1.5 MiB/  1.5 MiB] 100% Done                                    
Operation completed over 1 objects/1.5 MiB.                                      
CommandException: 1 file/object could not be transferred.
/bin/bash: line 1: Name:: command not found
Working with Zarzal - Valle del Cauca, corresponding with COD_MUNICIPIO 76895 and DPTO_COD 76
priority number: 77
Download images...
Copying gs://dym-indunor-temp/immap/v3_1/images/76/76895/76895_2021.tif...
\ [1/1 files][104.3 MiB/104.3 MiB] 100% Done                                    
Operation completed over 1 objects/104.3 MiB.                                    
gs://dym-indunor-temp/immap/v3_1/results/76/7

100%|████████████████████████████████████████████████████████████████| 52/52 [01:22<00:00,  1.58s/it]


Done!
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 3267, 2482
0...10...20...30...40...50...60...70...80...90...100 - done.
CommandException: No URLs matched: 77
Copying file://..//dataset/data_results/province_76895_76_year2021/100_100//../76895_2021.tif [Content-Type=image/tiff]...
/ [1/1 files][  2.6 MiB/  2.6 MiB] 100% Done                                    
Operation completed over 1 objects/2.6 MiB.                                      
CommandException: 1 file/object could not be transferred.
/bin/bash: line 1: Name:: command not found
